In [293]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import sys
import os

In [416]:
!pwd

/Users/bhavi/Documents/Projects/CristianoRonaldo


In [417]:
df = pd.read_csv("/Users/bhavi/Documents/Projects/CristianoRonaldo/Cristano_Ronaldo_Final_v1/data.csv")

In [418]:
# df.head()

In [419]:
submission = df[df.is_goal.isnull()]
# test_data['shot_id_number'].shape,test_data.index.shape
submission = submission.copy()
submission.loc[:, 'shot_id_number'] = submission.index + 1


In [420]:
submission[['shot_id_number']].head()

# test_data.tocsv("TestSet.csv",index=False)

,shot_id_number
0,1
7,8
16,17
19,20
21,22


In [421]:
df = df.rename(columns={'distance_of_shot.1': 'distance_of_shot_1'})

In [422]:
data = df[df.is_goal.notnull()]
df.shape, data.shape, submission.shape

((30697, 28), (24429, 28), (6268, 28))

In [423]:
# data.fillna("", inplace=True)

In [424]:
limitPer = len(df) * .1
df = df.dropna(thresh=limitPer,axis=1)

In [425]:
df.shape

(30697, 28)

In [426]:
for col in data.columns:
    print (f"'{col}'", end=',')

'Unnamed: 0','match_event_id','location_x','location_y','remaining_min','power_of_shot','knockout_match','game_season','remaining_sec','distance_of_shot','is_goal','area_of_shot','shot_basics','range_of_shot','team_name','date_of_game','home/away','shot_id_number','lat/lng','type_of_shot','type_of_combined_shot','match_id','team_id','remaining_min.1','power_of_shot.1','knockout_match.1','remaining_sec.1','distance_of_shot_1',

In [427]:
X_cols = ['location_x','location_y','remaining_min','power_of_shot'\
         ,'knockout_match','game_season','remaining_sec','distance_of_shot'\
         ,'area_of_shot','shot_basics','range_of_shot','date_of_game'\
          ,'home/away','shot_id_number','lat/lng','type_of_shot','type_of_combined_shot'\
          ,'match_id','team_id','remaining_min.1','power_of_shot.1','knockout_match.1'\
          ,'remaining_sec.1','distance_of_shot_1']

<h1>Handling Missing Data</h1>

<h3>Distance Parameters</h3>
<ol>
<li>location_x</li>
<li>location_y</li>
<li>distance_of_shot</li>
<li>area_of_shot</li>
<li>shot_basics</li>
<li>range_of_shot</li>
</ol>

In [428]:
# # data['distance_of_shot'].fillna(data['distance_of_shot.1'])
# data.distance_of_shot.fillna(value = data.distance_of_shot_1)
data[data.distance_of_shot.isnull()].shape

(1257, 28)

In [429]:
data['distance_of_shot'] = data.apply(
    lambda row: row['distance_of_shot_1'] if np.isnan(row['distance_of_shot']) else row['distance_of_shot'],
    axis=1
)

In [430]:
data[data.distance_of_shot.isnull()].shape

(78, 28)

In [431]:
def findArea(x):
    if type(x) == float and np.isnan(x):
        return x
    else:
        return x.split('(')[1].split(')')[0]

data["area_of_shot"] = data["area_of_shot"].apply(findArea)

In [432]:
data[data.area_of_shot.isnull()].shape

(1182, 28)

In [433]:
def processLowerRangeOfShot(row):
    range_of_shot = str(row['range_of_shot'])
    try:
        if "Less Than" in range_of_shot:
            return 0, 8
        elif "+" in range_of_shot:
            return 24, 32
        else:
            low = range_of_shot.split('-')[0]
            high = range_of_shot.split('-')[1][0:2]
            return low, high
    except Exception as e:
        return float('NaN'), float('NaN')
    
    

# data['lower_range'] = data.apply(processRangeOfShot, axis=1)
data['lower_range'], data['upper_range'] = zip(*data.apply(processLowerRangeOfShot, axis=1))

In [434]:
data[['range_of_shot','lower_range', 'upper_range']]

,range_of_shot,lower_range,upper_range
1,8-16 ft.,8,16
2,16-24 ft.,16,24
3,16-24 ft.,16,24
4,Less Than 8 ft.,0,8
5,8-16 ft.,8,16
6,Less Than 8 ft.,0,8
8,8-16 ft.,8,16
9,8-16 ft.,8,16
10,24+ ft.,24,32
11,16-24 ft.,16,24


In [435]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

sub = data[data.location_x.notnull()]
sub = sub[sub.location_y.notnull()][sub.distance_of_shot.notnull()][sub.lower_range.notnull()][sub.upper_range.notnull()]
X_train = sub[['location_y', 'distance_of_shot', 'lower_range', 'upper_range']]
y_train = sub[['location_x']]

sub = data[data.location_x.isnull()]
sub = sub[sub.location_y.notnull()][sub.distance_of_shot.notnull()][sub.lower_range.notnull()][sub.upper_range.notnull()]
X_test = sub[['location_y', 'distance_of_shot', 'lower_range', 'upper_range']]

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

linreg.fit(X_train, y_train)

predicted = linreg.predict(X_test)

print (data[data.location_x.isnull()].shape, predicted.shape)

print (predicted.mean(), data[data.location_x.notnull()]['location_x'].mean())

data.loc[data.location_x.isnull() & \
         data.location_y.notnull() & \
        data.distance_of_shot.notnull() & \
        data.lower_range.notnull() & \
        data.upper_range.notnull(), 'location_x'] = predicted 

print (data[data.location_x.isnull()].shape)


/Users/bhavi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/bhavi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


((1156, 30), (1025, 1))

(7.175145074434061, 7.480719553102807)

In [448]:
data.loc[data.location_x.isnull() & \
         data.location_y.notnull() & \
        data.distance_of_shot.notnull() & \
        data.lower_range.notnull() & \
        data.upper_range.notnull(), 'location_x'] = predicted 

In [449]:
data[data.location_x.isnull()].shape

(131, 30)

In [457]:
data[data.location_y == 196][['location_x', 'location_y']]

,location_x,location_y
26,-1.586074,196.0
640,98.000000,196.0
916,43.000000,196.0
1368,3.000000,196.0
1373,89.000000,196.0
1607,70.000000,196.0
2132,-50.000000,196.0
2185,-18.000000,196.0
2573,-1.836074,196.0
3250,-158.000000,196.0


In [400]:
# sub = data[data.location_x.isnull()]
# sub = sub[sub.location_y.notnull()][sub.distance_of_shot.notnull()][sub.lower_range.notnull()][sub.upper_range.notnull()]
# sub.shape

/Users/bhavi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(1025, 30)